In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv(r"/kaggle/input/emnist/emnist-balanced-train.csv", header= None)
test = pd.read_csv(r"/kaggle/input/emnist/emnist-balanced-test.csv", header= None)
print(train.shape)
print(test.shape)

In [48]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import cv2

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras as kr

In [49]:
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import layers
from keras.layers import *
from keras import utils
from sklearn.preprocessing import OneHotEncoder 

In [ ]:
test.head()

In [51]:
label_map = pd.read_csv("/kaggle/input/emnist/emnist-balanced-mapping.txt", 
                        delimiter = ' ', 
                        index_col=0, 
                        header=None, 
                        )
label_m = label_map.squeeze()

In [ ]:
label_dict2={}
for i,labels in enumerate(label_m):
    label_dict2[labels] = chr(labels)
label_dict2

In [ ]:
label_dict={}
for i,labels in enumerate(label_m):
    label_dict[i] = chr(labels)
label_dict

In [ ]:
x_train = train.iloc[:, 1:]
y_train = train.iloc[:, 0]

x_train.shape, y_train.shape

In [ ]:
x_test = test.iloc[:, 1:] 
y_test = test.iloc[:, 0]
x_test.shape, y_test.shape

In [56]:
sam_img=x_train.iloc[200]#this returns a series from the dataframe. Hence the shape is '(784,)'
sam_lab=y_train.iloc[200]

In [57]:
## A code to reshape a vector to matrix using numpy  

#"import numpy as np"
#(the lines in '' are the code lines).
# A flattened 1D array of length 784 (representing a 28x28 image).
#'flattened_image = np.arange(1, 785)  # Example array with values 1 to 784'

# Reshape it to 28x28.
#'reshaped_image = flattened_image.reshape(28, 28)'

#'print(reshaped_image)'

# ## A code to reshape a vector to matrix using numpy 

import numpy as np


**( A flattened 1D array of length 784 (representing a 28x28 image).)**

flattened_image = np.arange(1, 785)  

**(Example array with values 1 to 784)**


**(Reshape it to 28x28.)**
reshaped_image = flattened_image.reshape(28, 28)


print(reshaped_image)

In [ ]:
W=28
H=28
print("Label entry No. 200:", label_dict[sam_lab])
plt.imshow(sam_img.values.reshape(W,H), cmap="gray")
plt.show()

The images need to be preprocessed to be visualised properly. Hence we define a function to flip, rotate, and transform the images to grayscale.

In [59]:
def preproc_1(image):
    W = 28
    H = 28
    image = image.to_numpy(copy=True).reshape(W, H)#numpy methods work well with pandas dataframes and series(for this case).
    image = np.fliplr(image)
    image = np.rot90(image)
    plt.imshow(image,cmap="gray")
    plt.show()
    return image

In [ ]:
print("Label entry No. 200:", label_dict[sam_lab])
img=preproc_1(sam_img)

In [61]:
def preproc_all(image):
    W = 28
    H = 28
    image = image.reshape(W, H)#numpy methods work well with pandas dataframes and series(for this case).
    image = np.fliplr(image)
    image = np.rot90(image)
    return image

In [ ]:
x_train = np.apply_along_axis(preproc_all, 1, x_train)
x_train.shape

In [ ]:
x_test = np.apply_along_axis(preproc_all, 1, x_test)
x_test.shape

In [ ]:
plt.figure(figsize=(10, 8), dpi=80)
for i in range(100, 118):
    plt.subplot(3,6,i-99)
    plt.imshow(x_train[i], cmap="inferno")
    plt.title(label_dict[y_train[i]])

In [ ]:
plt.figure(figsize=(10, 8), dpi=80)
for i in range(100, 118):
    plt.subplot(3,6,i-99)
    plt.imshow(x_test[i], cmap="inferno")
    plt.title(label_dict[y_test[i]])

In [ ]:
x_train= x_train.astype(float)/255
x_test= x_test.astype(float)/255
plt.imshow(x_train[120],cmap="gray")
plt.title(label_dict[y_train[120]])
plt.show()
plt.imshow(x_test[120],cmap="inferno")
plt.title(label_dict[y_test[120]])
plt.show()

# **ONE-HOT ENCODING** 

For the ease of classification we will be one-hot encoding it to make it categorical in a binary format.
example-[1][0][0]....[0]x33 to represent 0.

In [ ]:
classes=  y_train.nunique()
classes

In [68]:
encoder = OneHotEncoder(sparse_output=False)
y_train= y_train.values.reshape(-1,1)
y_train = encoder.fit_transform(y_train)

In [69]:
encoder = OneHotEncoder(sparse_output=False)
y_test= y_test.values.reshape(-1,1)
y_test = encoder.fit_transform(y_test)

In [ ]:
y_train[120]

In [ ]:
y_test[120]

In [ ]:
print(x_train.shape)
x_train=x_train.reshape(-1,W,H,1)
print(x_train.shape)

In [ ]:
print(x_test.shape)
x_test=x_test.reshape(-1,W,H,1)
print(x_test.shape)

In [ ]:
x_cv,x_test,y_cv,y_test= train_test_split(x_test,y_test,test_size=0.4, random_state=88)
x_cv.shape, y_cv.shape

In [ ]:
model= Sequential([
kr.layers.Flatten(input_shape =(28,28),),
Dense(94, input_shape=(784,), activation='relu'),
Dense(94, activation='relu'),
Dense(47, activation='softmax')
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
hist=model.fit(x_train, y_train,validation_split=0.1, epochs=10)

In [ ]:
y_pred = model.predict(x_cv)
# Finding shape
y_pred.shape

In [ ]:
plt.figure(figsize=(10, 8), dpi=80)
for i in range(200, 218):
    plt.subplot(3,6,i-199)
    plt.imshow(x_cv[i], cmap=plt.cm.gray)
    plt.title(label_dict[y_pred[i].argmax()])

In [91]:
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
loss = hist.history['loss']
val_loss = hist.history['val_loss']
epochs = range(1,len(acc)+1)

In [92]:
def plotgraph(epochs, acc, val_acc):
    # Plot training & validation accuracy values
    plt.plot(epochs, acc, 'b')
    plt.plot(epochs, val_acc, 'r')
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

In [ ]:
plotgraph(epochs, acc, val_acc)